In [476]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

## Read the 2018_Chase_Loan_Cleaned.csv data from the export folder into a Pandas DataFrame.

In [477]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('export/2018_Chase_Loan_Cleaned_v2.csv')

df = pd.read_csv(data)

#using 2,3,5,6,7,8 results from  acition_taken and dropping 1 and 4.  
#also only home loan appplications - no re-fi or cash taken out
additional_drops_for_ML = ['activity_year','loan_type','loan_purpose', 'business_or_commercial_purpose', 'state_code']

#67% / 68% 
#additional_drops_for_ML = ['activity_year','loan_type','loan_purpose', 'business_or_commercial_purpose', 'state_code', 'construction_method', 'occupancy_type', 'total_units']

#66% / 67%
#additional_drops_for_ML = ['activity_year','loan_type','loan_purpose', 'business_or_commercial_purpose', 'state_code', 'construction_method', 'occupancy_type', 'total_units', 'county_code', 'property_value']

#using original data source
#
#additional_drops_for_ML = ['intro_rate_period', 'multifamily_affordable_units', 'prepayment_penalty_term','combined_loan_to_value_ratio','rate_spread', 'total_points_and_fees','co_applicant_race_2','co_applicant_race_3','co_applicant_race_4','co_applicant_race_5','applicant_race_2','applicant_race_3','applicant_race_4','applicant_race_5','co_applicant_ethnicity_2','co_applicant_ethnicity_3','co_applicant_ethnicity_4',
#'co_applicant_ethnicity_5','applicant_ethnicity_2','applicant_ethnicity_3','applicant_ethnicity_4','applicant_ethnicity_5','denial_reason_2', 'denial_reason_3', 'denial_reason_4', 'aus_1', 'aus_2', 'aus_3', 'aus_4', 'aus_5', 'lei', 'activity_year', 'state_code', 'debt_to_income_ratio', 'applicant_age', 'co_applicant_age', 'total_units', 'applicant_age_above_62', 'co_applicant_age_above_62']


df = df.drop(columns=additional_drops_for_ML)

# Review the DataFrame
df.head()

,action_taken,construction_method,occupancy_type,county_code,loan_amount,income,loan_term,property_value,total_units
0,1,1,1,53061,365000,139000.0,360.0,455000.0,1
1,1,1,1,48027,65000,48000.0,360.0,75000.0,1
2,1,1,1,15007,365000,62000.0,360.0,455000.0,1
3,1,1,1,36103,255000,74000.0,360.0,385000.0,1
4,1,1,1,48201,105000,35000.0,360.0,115000.0,1


In [478]:
#testing the data to see make sure its not garbage in - garbage out.
#export to CSV
#df.to_csv('export/where_is_bad_data.csv', index=False)


## Create the labels set (y) from the "action_taken" column, and then create the features (X) DataFrame from the remaining columns.

In [479]:
#y is the action taken results approved or denied applications
y = df['action_taken']

#X is the remaining columns
X = df.drop(columns='action_taken')

In [480]:
#display y
y

0        1
1        1
2        1
3        1
4        1
        ..
91444    1
91445    1
91446    1
91447    1
91448    1
Name: action_taken, Length: 91449, dtype: int64

In [481]:
#display x values
X

,construction_method,occupancy_type,county_code,loan_amount,income,loan_term,property_value,total_units
0,1,1,53061,365000,139000.0,360.0,455000.0,1
1,1,1,48027,65000,48000.0,360.0,75000.0,1
2,1,1,15007,365000,62000.0,360.0,455000.0,1
3,1,1,36103,255000,74000.0,360.0,385000.0,1
4,1,1,48201,105000,35000.0,360.0,115000.0,1
...,...,...,...,...,...,...,...,...
91444,1,1,29189,95000,52000.0,180.0,125000.0,1
91445,1,1,47037,165000,45000.0,360.0,175000.0,1
91446,1,1,29113,105000,49000.0,120.0,215000.0,1
91447,1,1,29189,505000,282000.0,360.0,635000.0,1


## Check the balance of the labels variable (y).

In [482]:
y.value_counts()


1    76760
0    14689
Name: action_taken, dtype: int64

## Split the data into training and testing datasets by using train_test_split.

In [483]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(68586, 8)

## Create a Logistic Regression Model with the Original Data
* Fit a logistic regression model by using the training data (X_train and y_train).

In [484]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [485]:
# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

## Save the predictions on the testing data labels by using the testing feature data (X_test) and the fitted model.

In [486]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

#tried to drop the indexes to see if it will map plot - did not help
#results = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

results.head(10)

,Prediction,Actual
0,1,1
1,1,1
2,1,0
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,0
9,1,1


## Evaluate the model’s performance 

In [487]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score

# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.8510694134628002

In [488]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8510920596040008
Testing Data Score: 0.8510694134628002


In [489]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  268,  3404],
       [    1, 19190]], dtype=int64)

In [490]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ["Approved", "Denied"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

    Approved       1.00      0.07      0.14      3672
      Denied       0.85      1.00      0.92     19191

    accuracy                           0.85     22863
   macro avg       0.92      0.54      0.53     22863
weighted avg       0.87      0.85      0.79     22863



## Predict a Logistic Regression Model with Resampled Training Data

In [497]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_sampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_ran, y_ran = random_sampler.fit_resample(X_train, y_train)

In [498]:
# Count the distinct values of the resampled labels data
y_ran.value_counts()

0    57569
1    57569
Name: action_taken, dtype: int64

## Use the LogisticRegression classifier and the resampled data to fit the model and make predictions.

In [499]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
re_classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)


# Fit the model using the resampled training data
re_classifier.fit(X_ran, y_ran)

# Make a prediction using the testing data
re_predictions = re_classifier.predict(X_test)

results = pd.DataFrame({"Prediction": re_predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,1,1
1,0,1
2,1,0
3,1,1
4,1,1
5,0,1
6,0,1
7,1,1
8,0,0
9,1,1


## Evaluate the resampled data

In [500]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, re_predictions)

0.5802862853059867

In [501]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, re_predictions)

array([[ 1739,  1933],
       [ 6007, 13184]], dtype=int64)

In [502]:
# Print the classification report for the model
target_names = ["Approved", "Denied"]
print(classification_report(y_test, re_predictions, target_names=target_names))

              precision    recall  f1-score   support

    Approved       0.22      0.47      0.30      3672
      Denied       0.87      0.69      0.77     19191

    accuracy                           0.65     22863
   macro avg       0.55      0.58      0.54     22863
weighted avg       0.77      0.65      0.69     22863

